# Recommendation : Collaborative Filtering

In [1]:
import pandas as pd
import numpy as np
import scipy as sp

In [2]:
import os, sys

In [3]:
print( os.curdir )
print( os.getcwd() )
print( os.path.abspath( os.getcwd() ) )

.
/Users/nururrahman/myRecommend
/Users/nururrahman/myRecommend


In [4]:
dataPath = os.path.abspath( os.getcwd() ) + '/Data/retailRocket/'
data_event  = pd.read_csv( dataPath + 'events.csv')

In [5]:
print( data_event.shape )
print( len( data_event.visitorid.unique() ) )
print( len( data_event.itemid.unique() ) )
print( data_event.event.unique() ) 

(2756101, 5)
1407580
235061
['view' 'addtocart' 'transaction']


In [6]:
data_event.groupby(['event']).agg({'count'})

,timestamp,visitorid,itemid,transactionid
,count,count,count,count
event,,,,
addtocart,69332,69332,69332,0
transaction,22457,22457,22457,22457
view,2664312,2664312,2664312,0


In [60]:
trans1 = data_event[ data_event.event =='transaction' ].reset_index(drop=True)
trans1.shape

(22457, 5)

In [61]:
visitors = trans1.visitorid.unique()
items = trans1.itemid.unique()
print( visitors.shape )
print( items.shape )

(11719,)
(12025,)


In [62]:
trans2 = trans1.groupby('visitorid').head(50)
trans2.shape

(19939, 5)

In [63]:
# trans2.groupby('visitorid').head()

In [64]:
tmp = trans2.copy()
tmp['visitors'] = tmp['visitorid'].apply( lambda row : np.argwhere(visitors==row)[0][0] )
trans2 = tmp

tmp = trans2.copy()
tmp['items'] = tmp['itemid'].apply( lambda row : np.argwhere(items==row)[0][0] )
trans2 = tmp

In [65]:
x1 = np.arange( trans2['visitorid'].unique().shape[0] )
x2 = np.sort( trans2['visitorid'].unique()  )
x  = np.column_stack((x1,x2))

In [66]:
from scipy.sparse import csr_matrix
visitor_item = csr_matrix( (visitors.shape[0], items.shape[0]), dtype='int8')

def set_utility_matrix(visitor, item):
    visitor_item[visitor, item] +=1

trans2.apply( lambda row : set_utility_matrix(row['visitors'], row['items']), axis=1)

cooc = visitor_item.T.dot(visitor_item)
cooc.setdiag = 0
cooc.toarray()[0:20, 0:15]

array([[ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  3,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 44,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0

In [67]:
visitor_item.toarray().max()

5

### Method 2

In [68]:
trans_count = trans2.groupby(['visitorid','itemid']).size().reset_index(drop=False)
trans_count = trans_count.rename(columns={0:'count'})
trans_count.head(3)

,visitorid,itemid,count
0,172,10034,1
1,172,465522,1
2,186,49029,1


In [69]:
utility = trans_count.pivot(index='visitorid', columns='itemid', values='count')
utility.columns.name = ''
utility.index.name = ''
utility.head(3)

,15,19,25,42,147,199,212,233,304,320,...,466301,466319,466321,466342,466443,466464,466526,466614,466710,466861
,,,,,,,,,,,,,,,,,,,,,
172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
utility = utility.fillna( value=0, inplace=False)
utility.head(5)

,15,19,25,42,147,199,212,233,304,320,...,466301,466319,466321,466342,466443,466464,466526,466614,466710,466861
,,,,,,,,,,,,,,,,,,,,,
172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Check

In [71]:
utility.values.max()

5.0

In [72]:
s = utility.max(axis=1)
#print( s[ s==s.max() ] )
maxind_ax1 = s[ s==s.max()].index[0]

s = utility.max(axis=0)
#print( s[ s==s.max() ] )
maxind_ax0 = s[ s==s.max()].index[0]

print( maxind_ax1, maxind_ax0 )

660393 224549


In [73]:
inds = utility.index.to_numpy()
cols = utility.columns.to_numpy()
max_row_idx = inds[ np.isin( inds, np.array(maxind_ax1) ) ]
max_col_idx = cols[ np.isin( cols, np.array(maxind_ax0) ) ]
print(max_row_idx, max_col_idx)

[660393] [224549]


In [74]:
s = utility[max_col_idx[0]]
s[ s > 1 ].sort_values(ascending=False).head(5)


660393    5.0
Name: 224549, dtype: float64

In [75]:
tmp = data_event[(data_event.visitorid==maxind_ax1) & (data_event.event=='transaction')]
print( tmp.shape )
print( tmp.itemid.nunique() )
tmp.groupby(['itemid']).size().sort_values(ascending=False).head(5)

(7, 5)
3


itemid
224549    5
398141    1
322424    1
dtype: int64

In [76]:
# nonzero_index = np.nonzero(utility.values)
# nonzero_vals  = utility.values[  nonzero_index ]
# print(nonzero_index)
# np.flip( np.sort(nonzero_vals) )
# pd.DataFrame({'x':np.flip( np.sort(non_zero) )}).clip(lower=0, upper=5)

In [85]:
utility = utility.clip( lower=0, upper=1 )

In [86]:
utility_matrix = utility.values
cooc = np.dot(utility_matrix.T, utility_matrix)
cooc.shape

(11085, 11085)

In [87]:
np.diag(cooc, k=0)[0:20]

array([ 1.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,
       17.,  1.,  1.,  1.,  1.,  1.,  1.])

In [ ]:
# print( cooc.sum() )
# cooc.setdiag = 10
# print( cooc.sum() )
# cooc.toarray()

In [ ]:
def xLogX(val):
    if val > 0.0:
        y = val * np.log(val) 
    else: 
        y = 0.0
    return y


def entropy(x1, x2=0, x3=0, x4=0):
    return xLogX(x1 + x2 + x3 + x4) - xLogX(x1) - xLogX(x2) - xLogX(x3) - xLogX(x4)


def LLR(k11, k12, k21, k22):
    rowEntropy = entropy(k11 + k12, k21 + k22)
    colEntropy = entropy(k11 + k21, k12 + k22)
    matrixEntropy = entropy(k11, k12, k21, k22)
    
    if rowEntropy + colEntropy < matrixEntropy:
        return 0.0
    
    return 2.0 * (rowEntropy + colEntropy - matrixEntropy)


def rootLLR(k11, k12, k21, k22):
    llr = LLR(k11, k12, k21, k22)
    sqrt = np.sqrt(llr)
    
    if k11 * 1.0 / (k11 + k12) < k21 * 1.0 / (k21 + k22):
        sqrt = -sqrt
    
    return sqrt

In [ ]:
row_sum  = np.sum(cooc, axis=0).A.flatten()
col_sum  = np.sum(cooc, axis=1).A.flatten()
total    = np.sum(row_sum, axis=0)

pp_score = csr_matrix((cooc.shape[0], cooc.shape[1]), dtype='double')

cx = cooc.tocoo()

for i,j,v in zip(cx.row, cx.col, cx.data):
    if v != 0:
        k11 = v
        k12 = row_sum[i] - k11
        k21 = col_sum[j] - k11
        k22 = total - k11 - k12 - k21
        pp_score[i,j] = rootLLR(k11, k12, k21, k22)

In [ ]:
np.log(0.0)

In [ ]:
result_value = np.flip(np.sort(pp_score.A, axis=1), axis=1)
result_index = np.flip(np.argsort(pp_score.A, axis=1), axis=1)

In [ ]:
result_value[8456]

In [ ]:
result_index[8456]

In [ ]:
minLLR = 5

indicators = result_value[:, :50]

indicators[indicators < minLLR] = 0.0

indicators_indices = result_index[:, :50]

max_indicator_indices = (indicators==0).argmax(axis=1)
themax = max_indicator_indices.max()

indicators = indicators[:, :themax+1]
indicators_indices = indicators_indices[:, :themax+1]

In [ ]:
indicators_indices

In [ ]:
indicators_indices.shape

In [ ]:
import requests
import json

In [ ]:
actions = []

for i in range(indicators.shape[0]):
    
    length = indicators[i].nonzero()[0].shape[0]
    
    real_indicators = items[indicators_indices[i, :length]].astype("int").tolist()
    id = items[i]
    
    action = { "index" : { "_index" : "items2", "_id" : str(id) } }
    
    data = {
        "id": int(id),
        "indicators": real_indicators
    }
    
    actions.append(json.dumps(action))
    actions.append(json.dumps(data))
    
    if len(actions) == 200:
        actions_string = "\n".join(actions) + "\n"
        actions = []
        
        url = "http://127.0.0.1:9200/_bulk/"
        headers = {"Content-Type" : "application/x-ndjson"}
        requests.post(url, headers=headers, data=actions_string)
    
    
    if len(actions) > 0:
        actions_string ="\n".join(actions) + "\n"
        actions = []    
        
        url = "http://127.0.0.1:9200/_bulk/"
        headers = {"Content-Type" : "application/x-ndjson"}
        requests.post(url, headers=headers, data=actions_string)